In [1]:
"""
Bar-Joseph Lab Rotation
Caleb Ellington
10/14/20

SNARE-seq transcription factor analysis workflow
"""

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import mmread
from scipy.stats import ttest_ind, linregress

/Users/calebellington/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# Build a dictionary of {gene_id: (chr#, start, stop)}
# Using http://hgdownload.soe.ucsc.edu/goldenPath/mm10/bigZips/genes/mm10.ncbiRefSeq.gtf.gz
ref = pd.read_csv("reference_genomes/mm10.ncbiRefSeq.gtf", sep="\t", header=None)
genes_ref = {}

# trim reference
ref = ref[ref[2] == 'transcript']

for idx, row in ref.iterrows():
    gene = row[8].split("\"")[1]
    chrnum = row[0]
    start = row[3]
    stop = row[4]
    if gene not in genes_ref:
        genes_ref[gene] = (chrnum, start, stop)

In [3]:
# Get a list of SNAREseq peaks
# Using AdBrainCortex from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE126074
peaks_data = pd.read_csv("SNAREseq_data/GSE126074_AdBrainCortex_SNAREseq_chromatin.peaks.tsv", sep="\t", header=None)
peaks = []

for idx, row in peaks_data.iterrows():
    chrnum, indices = row[0].split(":")
    start, stop = indices.split("-")
    start = int(start)
    stop = int(stop)
    peaks.append((chrnum, start, stop))

In [4]:
# Get a list of SNAREseq genes
# Using AdBrainCortex from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE126074
genes_data = pd.read_csv("SNAREseq_data/GSE126074_AdBrainCortex_SNAREseq_cDNA.genes.tsv", sep="\t", header=None)
genes = []

for idx, row in genes_data.iterrows():
    gene = row[0]
    if gene in genes_ref.keys():  # Excludes ~11000 genes when using mm10
        genes.append((idx, gene))

In [5]:
# Create a chrnum lookup for peaks and genes
chr_gene = {}
chr_peak = {}
for g_i, gene in genes:
    chrnum, start, stop = genes_ref[gene]
    if chrnum not in chr_gene.keys():
        chr_gene[chrnum] = []
    chr_gene[chrnum].append((gene, start, stop))
for p_i, peak in enumerate(peaks):
    chrnum, start, stop = peak
    if chrnum not in chr_peak.keys():
        chr_peak[chrnum] = []
    chr_peak[chrnum].append((p_i, start, stop))

In [6]:
# O(n^2) approach won't work, genes*peaks = 8109079040
# TODO: Depreciated reference genome, update to GRCm38

# Sort by start location
for chrnum in chr_peak.keys():
    chr_gene[chrnum].sort(key=lambda val:val[1])
    chr_peak[chrnum].sort(key=lambda val:val[1])

# O(n+m) pass
# 1. Pass over sorted genes for each chromosome
# 2. Increment the start/end index of the valid peaks
peaks_by_gene = {}
max_dist = 100000

# single pass over genes, 2 passes over peaks, O(m+2n)
for chrnum in chr_peak.keys():
    chr_genes = chr_gene[chrnum]
    chr_peaks = chr_peak[chrnum]
    frame_start, frame_end = 0, 1
    for gene, g_start, g_stop in chr_genes: 
        min_i = g_start - max_dist
        max_i = g_stop + max_dist
        while frame_start < len(chr_peaks) and min_i > chr_peaks[frame_start][1]:
            frame_start += 1
        frame_end = max(frame_end, frame_start+1)
        while frame_end < len(chr_peaks) and max_i > chr_peaks[frame_end][2]:
            frame_end += 1
        peaks_by_gene[gene] = chr_peaks[frame_start:frame_end]

In [7]:
# Load the cell-by-gene and cell-by-peak matrices
genecells = mmread("SNAREseq_data/GSE126074_AdBrainCortex_SNAREseq_cDNA.counts.mtx").toarray()
peakcells = mmread("SNAREseq_data/GSE126074_AdBrainCortex_SNAREseq_chromatin.counts.mtx").toarray()

In [8]:
# Get the correction value for multiple-hypothesis tests
correction = 0
for g_peaks in peaks_by_gene.values():
    correction += len(g_peaks)
correction

668613

In [9]:
# Get all significant peaks using multiple-hypothesis 2-sided t-test
sig_peaks = []
sig_pairs = []
for i in range(len(genes)):
    g_idx, gene = genes[i]
    if gene not in peaks_by_gene.keys():
        continue
    g_peaks = peaks_by_gene[gene]
    peak_ids = [idval for idval, _, _ in g_peaks]
    expr_vals = genecells[g_idx]
    open_vals = peakcells[peak_ids]
    
    for i, peak in enumerate(open_vals):
        open_cells = peak == 1
        if (open_cells).all() or (~open_cells).all():
            continue
        expr_open = expr_vals[open_cells]
        expr_closed = expr_vals[~open_cells]
        tstat, pval = ttest_ind(expr_open, expr_closed)
        if correction*pval < 0.05:
#             print(f"pval: {correction*pval}, peak: {g_peaks[i]}, gene: {gene, g_idx}")
            sig_peaks.append(peak_ids[i])
            sig_pairs.append((g_idx, peak_ids[i]))

/Users/calebellington/.pyenv/versions/3.7.3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3584: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/calebellington/.pyenv/versions/3.7.3/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
# Found 1686 peaks with significant impact on expression
# 1669 unique peak ids
# Get all peak sequences using genome.ucsd.edu
from urllib.request import urlopen
from xml.etree.ElementTree import parse

sig_peaks_unique = np.unique(sig_peaks)
peak_seqs = []
for peak_id in sig_peaks_unique:
    peak = peaks[peak_id]
    docurl = urlopen(f'http://genome.ucsc.edu/cgi-bin/das/mm10/dna?segment={peak[0]}:{peak[1]},{peak[2]}')
    seq = parse(docurl).getroot()[0][0].text
    seq = seq.replace('\n', '')
    peak_seqs.append(seq)

In [21]:
# Write peak seqs to .fasta for upload to meme-suite.org
with open("sig_peaks.fasta", "w") as fasta:
    for i, (peak_id, peak_seq) in enumerate(zip(sig_peaks_unique, peak_seqs)):
        fasta.write(f">sigpeak{i}: ({peaks[peak_id][0]}, {peaks[peak_id][1]}, {peaks[peak_id][2]})\n")
        fasta.write(peak_seq)
        fasta.write('\n')
    fasta.write('\n')
    fasta.close()

In [11]:
# Do meme-suite runs to get motifs, motif-binding TFs, and peaks w motifs
# 
# MEME motif analysis
# http://meme-suite.org/tools/meme
#
# Tomtom for matching transcription factors to to motifs
# http://meme-suite.org/tools/tomtom
#
# FIMO for finding motif locations in sig_peaks.fasta
# http://meme-suite.org/tools/fimo

motif1_tf_genes = ['Maz', 'Sp5']
motif2_tf_genes = ['Rreb1']
motif3_tf_genes = ['Sall1', 'Maz']
motif4_tf_genes = ['Stat1']
motif5_tf_genes = ['Ctcf', 'Ctcfl']
motifs = [motif1_tf_genes, motif2_tf_genes, motif3_tf_genes, motif4_tf_genes, motif5_tf_genes]

In [12]:
# Test motif-gene correlations for significant motif-having peaks

tf_gene_pairs = []
correction = 8 * len(sig_pairs)  # multiple hypothesis correction
for motif_id in range(len(motifs)):
    for tf_id in range(len(motifs[motif_id])):
        motif_peak_df = pd.read_csv(f'motif_peak_matches/fimo_motif{motif_id+1}.tsv', sep='\t')
        motif_peak_ids = motif_peak_df['sequence_name'].dropna().map(lambda x: int(x[7:]))

        tf_gene = motifs[motif_id][tf_id]
        tf_gene_idx = [idx for idx, gene in genes if gene == tf_gene][0]
        tf_expression = genecells[tf_gene_idx]

        correlation = []
        peaks_considered = 0
        for g_idx, p_idx in sig_pairs:
            if p_idx not in motif_peak_ids:
                continue
            motif_gene_expression = genecells[g_idx]
            motif_peak_open = peakcells[p_idx].astype(bool)
            if motif_peak_open.any() == False:
                continue
            open_gene_expr = motif_gene_expression[motif_peak_open]
            open_tf_expr = tf_expression[motif_peak_open]
            open_tf_active = np.logical_or(open_tf_expr != 0, open_gene_expr != 0)
            if open_tf_active.any() == False:
                continue
            peaks_considered += 1
            active_open_gene_expr = open_gene_expr[open_tf_active]
            active_open_tf_expr = open_tf_expr[open_tf_active]
            _, _, rval, pval, stderr = linregress(active_open_gene_expr, active_open_tf_expr)
            if not np.isnan(pval) and pval*correction < 0.05:
                inverse = ~np.logical_and(active_open_gene_expr, active_open_tf_expr).all()
                numdp = len(active_open_gene_expr)
                tf_gene_pairs.append((tf_gene, genes_data[0][g_idx], rval**2, pval, inverse, numdp))

/Users/calebellington/.pyenv/versions/3.7.3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/Users/calebellington/.pyenv/versions/3.7.3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:140: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/calebellington/.pyenv/versions/3.7.3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: invalid value encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [23]:
snareseq_tfs = pd.DataFrame(data=np.unique(tf_gene_pairs, axis=0), columns=['tf', 'gene', 'rsquared', 'pvalue', 'repressor', 'samples'])
snareseq_tfs.to_csv('snareseq_tfs.csv', index=False)
snareseq_tfs.head()

,tf,gene,rsquared,pvalue,repressor,samples
0,Maz,4930439A04Rik,1.0,9.003163161571059e-11,True,3
1,Maz,9130227L01Rik,1.0,9.003163161571059e-11,True,3
2,Maz,Aox3,1.0,1.2004217548761408e-30,True,5
3,Maz,Gm11578,1.0,0.0,True,2
4,Maz,Gm16150,1.0,9.003163161571059e-11,True,3
